<a href="https://colab.research.google.com/github/Maram-Elsayed/Airbnb-New-User-Bookings-Kaggle/blob/master/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

import statsmodels.api as sm
#from statsmodels.stats import diagnostic as diag
#from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer




In [0]:
# Read train and test data
df_train = pd.read_csv('drive/My Drive/airbnb/train_users_2.csv')
df_test = pd.read_csv('drive/My Drive/airbnb/test_users.csv')

In [0]:
# Remove the label from the train data 
train_labels= pd.DataFrame()
train_labels['country_destination']= df_train['country_destination']
df_train = df_train.drop(['country_destination'], axis=1)
piv_train = df_train.shape[0]

In [0]:
# Creating a DataFrame with train+test data
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

# Dropping column not available in test data
df_all = df_all.drop('date_first_booking', axis=1)

In [0]:
# Replace unknown languge values with "en"  
df_all.loc[df_all['language'] == '-unknown-', 'language'] = "en"

In [0]:
# Use LabelEncoder to change non numeric values to numeric 
le_language = LabelEncoder()
le_signup_method = LabelEncoder()
le_affiliate_channel = LabelEncoder()
le_affiliate_provider = LabelEncoder()
le_first_affiliate_tracked = LabelEncoder()
le_first_device_type = LabelEncoder()
le_first_browser = LabelEncoder()
le_signup_app=LabelEncoder()

In [0]:
df_all['language_n']=pd.DataFrame({'Column1':le_language.fit_transform(df_all['language'])})
df_all['affiliate_channel_n']=pd.DataFrame({'Column1':le_language.fit_transform(df_all['affiliate_channel'])})
df_all['signup_method_n']=pd.DataFrame({'Column1':le_language.fit_transform(df_all['signup_method'])})
df_all['affiliate_provider_n']=pd.DataFrame({'Column1':le_language.fit_transform(df_all['affiliate_provider'])})
df_all['first_device_type_n']=pd.DataFrame({'Column1':le_language.fit_transform(df_all['first_device_type'])})
df_all['first_browser_n']=pd.DataFrame({'Column1':le_language.fit_transform(df_all['first_browser'])})
df_all['signup_app_n']=pd.DataFrame({'Column1':le_language.fit_transform(df_all['signup_app'])})

In [0]:
# Replace first_affiliate_tracked unknown values to "untracked" and changing it to numeric
df_all.loc[ df_all['first_affiliate_tracked'].isnull(), 'first_affiliate_tracked'] = "untracked"
df_all['first_affiliate_tracked_n']=pd.DataFrame({'Column1':le_first_affiliate_tracked.fit_transform(df_all['first_affiliate_tracked'])})

In [48]:
# Drop non numeric columns
df_all=df_all.drop(['signup_method','language','affiliate_channel','affiliate_provider','first_device_type','first_browser','first_affiliate_tracked','signup_app'],axis=1)

df_all.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_flow,language_n,affiliate_channel_n,signup_method_n,affiliate_provider_n,first_device_type_n,first_browser_n,signup_app_n,first_affiliate_tracked_n
0,gxn3p5htnn,2010-06-28,20090319043255,-unknown-,NaN,0,5,2,1,4,3,8,2,6
1,820tgsjxq7,2011-05-25,20090523174809,MALE,38.0,0,5,7,1,8,3,8,2,6
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,3,5,2,0,4,6,22,2,6
3,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,42.0,0,5,2,1,4,3,17,2,6
4,87mebub9p4,2010-09-14,20091208061105,-unknown-,41.0,0,5,2,0,4,3,8,2,6


In [0]:
# Replacing unknown values in gender with -1 and null values with -1
df_all.loc[ df_all['gender'] == '-unknown-', 'gender'] = -1
df_all.loc[ df_all['gender'].isnull(), 'gender' ] = -1


In [0]:
# Encoding Female with 0, Male with 1 and Other with 2 in both test and train data
gender_translation = {'FEMALE' : 0,
                     'MALE' : 1,
                     'OTHER' : 2,
                     -1 : -1 }
for data in [df_all]:
    data['gender'] = data['gender'].apply(lambda x: gender_translation[x])


In [51]:
# Finding valid values for gender and invalid values for gender
nan_gender_count = len(df_all.loc[df_all['gender'] == -1, 'gender'])
valid_gender_count = len(df_all.gender.values) - nan_gender_count

# Creating a map with the gender distribution
count_map = pd.value_counts(df_all['gender'].values)
print ("Existing gender value distribution")
for k, v in count_map.iteritems():
    if k == -1:
        continue
    print (k, ":", float(v)/float(valid_gender_count))


Existing gender value distribution
0 : 0.5307427413447253
1 : 0.46697063676258155
2 : 0.002286621892693079


In [52]:
# Making the gender distribution the same for missing imputation
for k, v in count_map.iteritems():
    if k == -1:
        continue
    c = int ( nan_gender_count * float(v)/float(valid_gender_count) )
    for i in range(len(df_all.gender.values)):
        if df_all.gender.values[i] == -1:
            df_all.gender.values[i] = k
            c -= 1
        if c == 0:
            break
df_all.gender.values[275545] = 0
df_all['gender'].describe()

count    275547.000000
mean          0.471542
std           0.503750
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           2.000000
Name: gender, dtype: float64

In [53]:
# Display 'age' distribution
df_all['age'].describe()

count    158681.000000
mean         47.145310
std         142.629468
min           1.000000
25%          28.000000
50%          33.000000
75%          42.000000
max        2014.000000
Name: age, dtype: float64

In [54]:
# Replacing invalid age with NaN in test and train
df_all.loc[df_all['age'] > 100, 'age'] = np.nan
df_all.loc[df_all['age'] < 15, 'age'] = np.nan

# Replace missing age with median
print (df_all.age.median())
df_all.loc[ df_all['age'].isnull(), 'age' ] = df_all.age.median()
df_all['age'].describe()

33.0


count    275547.000000
mean         34.728848
std           8.903830
min          15.000000
25%          32.000000
50%          33.000000
75%          34.000000
max         100.000000
Name: age, dtype: float64

In [55]:
df_all.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_flow,language_n,affiliate_channel_n,signup_method_n,affiliate_provider_n,first_device_type_n,first_browser_n,signup_app_n,first_affiliate_tracked_n
0,gxn3p5htnn,2010-06-28,20090319043255,0,33.0,0,5,2,1,4,3,8,2,6
1,820tgsjxq7,2011-05-25,20090523174809,1,38.0,0,5,7,1,8,3,8,2,6
2,4ft3gnwmtx,2010-09-28,20090609231247,0,56.0,3,5,2,0,4,6,22,2,6
3,bjjt8pjhuk,2011-12-05,20091031060129,0,42.0,0,5,2,1,4,3,17,2,6
4,87mebub9p4,2010-09-14,20091208061105,0,41.0,0,5,2,0,4,3,8,2,6


In [57]:
# Date is split into 3 parts as year, month and day in both test and train. These are added as
# new features in both test and train

date_account_created = np.vstack(df_all.date_account_created.astype(str).apply(
        lambda x: list(map(int, x.split('-')))).values)
df_all['created_year'] = date_account_created[:,0]
df_all['created_month'] = date_account_created[:,1]
df_all['created_day'] = date_account_created[:,2]
df_all = df_all.drop(['date_account_created'], axis=1)
df_all.head()

,id,timestamp_first_active,gender,age,signup_flow,language_n,affiliate_channel_n,signup_method_n,affiliate_provider_n,first_device_type_n,first_browser_n,signup_app_n,first_affiliate_tracked_n,created_year,created_month,created_day
0,gxn3p5htnn,20090319043255,0,33.0,0,5,2,1,4,3,8,2,6,2010,6,28
1,820tgsjxq7,20090523174809,1,38.0,0,5,7,1,8,3,8,2,6,2011,5,25
2,4ft3gnwmtx,20090609231247,0,56.0,3,5,2,0,4,6,22,2,6,2010,9,28
3,bjjt8pjhuk,20091031060129,0,42.0,0,5,2,1,4,3,17,2,6,2011,12,5
4,87mebub9p4,20091208061105,0,41.0,0,5,2,0,4,3,8,2,6,2010,9,14


In [0]:
# Reading sessions data
sessions = pd.read_csv('drive/My Drive/airbnb/sessions.csv')

In [0]:
# frequency of each user_id in sessions data
df = sessions['user_id'].value_counts()

In [0]:
# Updating session_count for users present in the train data

df_all['session_count'] = 0

for key,val in df.iteritems():
    print(key,val)
    df_all.loc[df_all[ 'id' ] == key, 'session_count'] = val

In [140]:
# Divide data into train and test again after preprocessing in done
train=df[:piv_train]
test=df[piv_train:]
display(train.head())
display(test.head())

,id,timestamp_first_active,gender,age,signup_flow,language_n,affiliate_channel_n,signup_method_n,affiliate_provider_n,first_device_type_n,first_browser_n,signup_app_n,first_affiliate_tracked_n,session_count,created_year,created_month,created_day
0,gxn3p5htnn,20090319043255,0,33.0,0,5,2,1,4,3,8,2,6,0.0,2010,6,28
1,820tgsjxq7,20090523174809,1,38.0,0,5,7,1,8,3,8,2,6,0.0,2011,5,25
2,4ft3gnwmtx,20090609231247,0,56.0,3,5,2,0,4,6,22,2,6,0.0,2010,9,28
3,bjjt8pjhuk,20091031060129,0,42.0,0,5,2,1,4,3,17,2,6,0.0,2011,12,5
4,87mebub9p4,20091208061105,0,41.0,0,5,2,0,4,3,8,2,6,0.0,2010,9,14


,id,timestamp_first_active,gender,age,signup_flow,language_n,affiliate_channel_n,signup_method_n,affiliate_provider_n,first_device_type_n,first_browser_n,signup_app_n,first_affiliate_tracked_n,session_count,created_year,created_month,created_day
213451,5uwns89zht,20140701000006,0,35.0,0,5,2,1,4,8,30,1,6,8.0,2014,7,1
213452,jtl0dijy2j,20140701000051,1,33.0,0,5,2,0,4,8,30,1,6,19.0,2014,7,1
213453,xx0ulgorjt,20140701000148,1,33.0,0,5,2,0,4,6,8,2,0,58.0,2014,7,1
213454,6c6puo6ix0,20140701000215,1,33.0,0,5,2,0,4,6,22,2,0,11.0,2014,7,1
213455,czqhjk3yfe,20140701000305,1,33.0,0,5,2,0,4,3,43,2,6,19.0,2014,7,1


In [141]:
# Append "country_destination" to train data again
train['country_destination']=0
train['country_destination']=train_labels['country_destination']
train.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,timestamp_first_active,gender,age,signup_flow,language_n,affiliate_channel_n,signup_method_n,affiliate_provider_n,first_device_type_n,first_browser_n,signup_app_n,first_affiliate_tracked_n,session_count,created_year,created_month,created_day,country_destination
0,gxn3p5htnn,20090319043255,0,33.0,0,5,2,1,4,3,8,2,6,0.0,2010,6,28,NDF
1,820tgsjxq7,20090523174809,1,38.0,0,5,7,1,8,3,8,2,6,0.0,2011,5,25,NDF
2,4ft3gnwmtx,20090609231247,0,56.0,3,5,2,0,4,6,22,2,6,0.0,2010,9,28,US
3,bjjt8pjhuk,20091031060129,0,42.0,0,5,2,1,4,3,17,2,6,0.0,2011,12,5,other
4,87mebub9p4,20091208061105,0,41.0,0,5,2,0,4,3,8,2,6,0.0,2010,9,14,US


In [0]:
# Save data after Preprocessing
train.to_csv('drive/My Drive/airbnb/train_Preprocessed_2.csv',index=False)
test.to_csv('drive/My Drive/airbnb/test_Preprocessed_2.csv',index=False)

In [144]:
train = pd.read_csv('drive/My Drive/airbnb/train_Preprocessed_2.csv')
test = pd.read_csv('drive/My Drive/airbnb/test_Preprocessed_2.csv')
train.head()

,id,timestamp_first_active,gender,age,signup_flow,language_n,affiliate_channel_n,signup_method_n,affiliate_provider_n,first_device_type_n,first_browser_n,signup_app_n,first_affiliate_tracked_n,session_count,created_year,created_month,created_day,country_destination
0,gxn3p5htnn,20090319043255,0,33.0,0,5,2,1,4,3,8,2,6,0.0,2010,6,28,NDF
1,820tgsjxq7,20090523174809,1,38.0,0,5,7,1,8,3,8,2,6,0.0,2011,5,25,NDF
2,4ft3gnwmtx,20090609231247,0,56.0,3,5,2,0,4,6,22,2,6,0.0,2010,9,28,US
3,bjjt8pjhuk,20091031060129,0,42.0,0,5,2,1,4,3,17,2,6,0.0,2011,12,5,other
4,87mebub9p4,20091208061105,0,41.0,0,5,2,0,4,3,8,2,6,0.0,2010,9,14,US


In [0]:
# Scale the data
from sklearn import preprocessing

stdscaler = preprocessing.StandardScaler()

# Drop columns that shouldn't be scaled
train_without_id=train.drop(['id','country_destination'],axis=1)
test_without_id=test.drop('id',axis=1)

train_users_scaled = stdscaler.fit_transform(train_without_id.values)
test_users_scaled = stdscaler.fit_transform(test_without_id.values)

train_users = pd.DataFrame(train_users_scaled, columns = train_without_id.columns)
test_users = pd.DataFrame(test_users_scaled, columns = test_without_id.columns)
train_users['country_destination']=train_labels['country_destination']
train_users.insert(loc=0, column='id', value=train['id'])
test_users.insert(loc=0, column='id', value=test['id'])
display(train_users.head())
display(test_users.head())

In [0]:
# Save train and test data after scaling
train_users.to_csv('drive/My Drive/airbnb/train_scaled_2.csv',index=False)
test_users.to_csv('drive/My Drive/airbnb/test_scaled_2.csv',index=False)